In [1]:
from typing import Annotated

# from langchain_community.chat_models import ChatOllama
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import AIMessage, ToolMessage
from langchain_core.tools import tool
# from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from typing_extensions import TypedDict
from langchain_groq import ChatGroq

In [2]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    ask_human: bool


@tool
def request_assistance():
    """Escalate the conversation to an expert. Use this if the search query is ambiguous or unclear, prompt the user with a clarifying question to gather specific attributes or details about the product they are searching for.

    To use this function, relay the user's 'request' so the expert can provide the right guidance.
    """
    return ""


def chatbot(state: State):
    response = llm_with_tools.invoke(state["messages"])
    ask_human = False
    if response.tool_calls and response.tool_calls[0]["name"] == "request_assistance":
        ask_human = True
    return {"messages": [response], "ask_human": ask_human}


def create_response(response: str, ai_message: AIMessage):
    return ToolMessage(
        content=response,
        tool_call_id=ai_message.tool_calls[0]["id"],
    )


def human_node(state: State):
    new_messages = []
    if not isinstance(state["messages"][-1], ToolMessage):
        new_messages.append(
            create_response("No response from human.", state["messages"][-1])
        )
    return {
        "messages": new_messages,
        "ask_human": False,
    }


def select_next_node(state: State):
    if state["ask_human"]:
        return "human"
    return tools_condition(state)

In [3]:
tool = TavilySearchResults(max_results=2)
tools = [tool]
# llm = ChatOpenAI()
# llm = ChatOllama(model="llama3.1:8b")
llm = ChatGroq(
            model_name='llama3-8b-8192'
    )
llm_with_tools = llm.bind_tools(tools + [request_assistance])

In [4]:
graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=[tool]))
graph_builder.add_node("human", human_node)
graph_builder.add_conditional_edges(
    "chatbot",
    select_next_node,
    {"human": "human", "tools": "tools", "__end__": "__end__"},
)

graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("human", "chatbot")
graph_builder.set_entry_point("chatbot")
memory = SqliteSaver.from_conn_string(":memory:")

graph = graph_builder.compile(
    checkpointer=MemorySaver(),
    interrupt_before=["human"],
)

In [5]:
import uuid


def predict_react_agent_answer(example: dict):
    """Use this for answer evaluation"""

    config = {"configurable": {"thread_id": str(uuid.uuid4())}}
    messages = graph.invoke({"messages": ("user", example["input"])}, config)
    return {"response": messages["messages"][-1].content, "messages": messages}


example = {"input": "Hi, I am Rayen"}
response = predict_react_agent_answer(example)

In [6]:
response

{'response': 'Nice to meet you, Rayen!',
 'messages': {'messages': [HumanMessage(content='Hi, I am Rayen', id='05245acb-f0dd-42be-a097-6912e566057e'),
   AIMessage(content='Nice to meet you, Rayen!', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 1067, 'total_tokens': 1076, 'completion_time': 0.0075, 'prompt_time': 0.06214732, 'queue_time': 0.001660689, 'total_time': 0.06964732}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-8a772777-25d0-40ae-8d3f-5a5bfc6b6bb2-0', usage_metadata={'input_tokens': 1067, 'output_tokens': 9, 'total_tokens': 1076})],
  'ask_human': False}}